# 利用Python对链家网北京主城区二手房进行数据分析
* 本文主要讲述如何通过pandas对爬虫下来的链家数据进行相应的二手房数据分析，主要分析内容包括各个行政区，各个小区的房源信息情况。
* 数据来源 https://github.com/XuefengHuang/lianjia-scrawler 该repo提供了python程序进行链家网爬虫，并从中提取二手房价格、面积、户型和二手房关注度等数据。
* 分析方法参考 http://www.jianshu.com/p/44f261a62c0f

In [2]:
## 导入链家网二手房在售房源的文件（数据更新时间2018-06-15）

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import sys

stdout = sys.stdout

import importlib
importlib.reload(sys)

#reload(sys)
#sys.setdefaultencoding('utf-8')
sys.stdout = stdout

plt.rcParams['font.sans-serif'] = ['SimHei']    
plt.rcParams['axes.unicode_minus'] = False

#所有在售房源信息
house=pd.read_csv('houseinfo.csv')

# 所有小区信息
community=pd.read_csv('community.csv')

# 合并小区信息和房源信息表，可以获得房源更详细的地理位置
community['community'] = community['title']
house_detail = pd.merge(house, community, on='community')
house_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27019 entries, 0 to 27018
Data columns (total 32 columns):
houseID         27019 non-null int64
title_x         27019 non-null object
link_x          27019 non-null object
community       27019 non-null object
years           27019 non-null object
housetype_x     27019 non-null object
square          27019 non-null object
direction       27019 non-null object
floor           27019 non-null object
taxtype         25395 non-null object
totalPrice      27019 non-null float64
unitPrice       27019 non-null int64
followInfo      27019 non-null object
decoration      27019 non-null object
validdate_x     27019 non-null object
id              27019 non-null int64
title_y         27019 non-null object
link_y          27019 non-null object
district        27019 non-null object
bizcircle       27019 non-null object
tagList         20067 non-null object
onsale          27019 non-null int64
onrent          27019 non-null int64
year            26691

In [2]:
house_detail.head(2)

,houseID,title_x,link_x,community,years,housetype_x,square,direction,floor,taxtype,...,onrent,year,housetype_y,cost,service,company,building_num,house_num,price,validdate_y
0,101088604521,沁园公寓 三居室 距离苏州街地铁站383米,https://bj.lianjia.com/ershoufang/101088604521...,沁园公寓,低楼层(共24层)/1999年建塔楼/苏州桥,3室2厅,177.36平米,南 西 北,低楼层(共24层)/1999年建塔楼/苏州桥,距离10号线苏州街站388米房本满五年,...,1,1999年建成,塔楼,2.7元/平米/月,北京万达物业管理中心,北京大行基业房地产开发有限公司,1栋,192户,59736,2018-06-11 19:51:58
1,101101523892,人民大学北路33号2号楼 满五年高层大两居,https://bj.lianjia.com/ershoufang/101101523892...,沁园公寓,顶层(共24层)/1999年建塔楼/苏州桥,2室2厅,142.12平米,西南,顶层(共24层)/1999年建塔楼/苏州桥,房本满五年,...,1,1999年建成,塔楼,2.7元/平米/月,北京万达物业管理中心,北京大行基业房地产开发有限公司,1栋,192户,59736,2018-06-11 19:51:58


### Get house year , no info as 1900

In [39]:
# 将字符串转换成数字
def data_year(year_data, str):       
    if str in year_data :        
        return int(year_data[year_data.find(str)-4 : year_data.find(str)])    
    else :        
        return None

# 处理房屋面积数据
df=house_detail[['years','year']]
df['years'] = house_detail['years'].astype(str).apply(data_year,str = '年')
df['year'] = house_detail['year'].astype(str).apply(data_year,str = '年')
df['house_year']=df[['years']].fillna(df['year'],inplace=True)
df['house_year'].fillna(1900,inplace=True)
house_detail['house_year']=df['house_year'].astype('int',inplace=True)
house_detail['house_year']

/Users/laserk/anaconda/envs/Python36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/laserk/anaconda/envs/Python36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/laserk/anaconda/envs/Python36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

0        1999
1        1999
2        2007
3        2004
4        2005
5        2004
6        2007
7        2004
8        2004
9        2007
10       2004
11       2005
12       2004
13       2004
14       2005
15       2004
16       2007
17       2004
18       2006
19       2005
20       2005
21       2004
22       2005
23       2005
24       2004
25       2004
26       2007
27       2004
28       2004
29       2006
         ... 
26989    1999
26990    2011
26991    2010
26992    1998
26993    1986
26994    1986
26995    1988
26996    2000
26997    1992
26998    1994
26999    1900
27000    1980
27001    1989
27002    1995
27003    1988
27004    1988
27005    1980
27006    1995
27007    2016
27008    1985
27009    1988
27010    1996
27011    1985
27012    1986
27013    2007
27014    1999
27015    1991
27016    1994
27017    2002
27018    1976
Name: house_year, Length: 27019, dtype: int64

In [10]:
house_detail[['totalPrice','unitPrice','price']].head(5)

,totalPrice,unitPrice,price
0,1200.0,67659,59736
1,950.0,66845,59736
2,1650.0,67205,98032
3,780.0,121971,98032
4,1130.0,114385,98032


## 将数据从字符串提取出来

In [ ]:
# 将字符串转换成数字
def data_adj(area_data, str):       
    if str in area_data :        
        return float(area_data[0 : area_data.find(str)])    
    else :        
        return None
# 处理房屋面积数据
house['square'] = house['square'].apply(data_adj,str = '平米')
house['square'].head(2)

## 删除车位信息

In [ ]:
car=house[house.housetype.str.contains('车位')]
print ('记录中共有车位%d个'%car.shape[0])
house.drop(car.index,inplace=True)
print ('现在还剩下%d条记录'%house.shape[0])

## 价格最高的5个别墅

In [ ]:
bieshu=house[house.housetype.str.contains('别墅')]
print ('记录中共有别墅%d栋'%bieshu.shape[0])
bieshu.sort_values('totalPrice',ascending=False).head(5)

## 删除别墅信息

In [ ]:
house.drop(bieshu.index,inplace=True)
print ('现在还剩下%d条记录'%house.shape[0])

## 获取总价前五的房源信息

In [ ]:
house.sort_values('totalPrice',ascending=False).head(2)

## 获取户型数量分布信息

In [ ]:
housetype = house['housetype'].value_counts()
housetype.head(8).plot(kind='bar',x='housetype',y='size', title='户型数量分布')
plt.legend(['数量']) 
plt.show()

## 关注人数最多5套房子

In [ ]:
house['guanzhu'] = house['followInfo'].apply(data_adj,str = '人关注')
house.sort_values('guanzhu',ascending=False).head(5)

## 户型和关注人数分布

In [ ]:
fig, ax1 = plt.subplots(1,1)    
type_interest_group = house['guanzhu'].groupby(house['housetype']).agg([('户型', 'count'), ('关注人数', 'sum')])    
#取户型>50的数据进行可视化
ti_sort = type_interest_group[type_interest_group['户型'] > 50].sort_values(by='户型')    
ti_sort.plot(kind='barh', alpha=0.7, grid=True, ax=ax1)    
plt.title('二手房户型和关注人数分布')    
plt.ylabel('户型') 
plt.show()

## 面积分布

In [ ]:
fig,ax2 = plt.subplots(1,1)    
area_level = [0, 50, 100, 150, 200, 250, 300, 500]    
label_level = ['小于50', '50-100', '100-150', '150-200', '200-250', '250-300', '300-350']    
area_cut = pd.cut(house['square'], area_level, labels=label_level)        
area_cut.value_counts().plot(kind='bar', rot=30, alpha=0.4, grid=True, fontsize='small', ax=ax2)    
plt.title('二手房面积分布')    
plt.xlabel('面积')    
plt.legend(['数量'])    
plt.show()

## 聚类分析

In [ ]:
# 缺失值处理:直接将缺失值去掉    
cluster_data = house[['guanzhu','square','totalPrice']].dropna()    
#将簇数设为3    
K_model = KMeans(n_clusters=3)    
alg = K_model.fit(cluster_data)    
'------聚类中心------'   
center = pd.DataFrame(alg.cluster_centers_, columns=['关注人数','面积','房价'])    
cluster_data['label'] = alg.labels_ 
center

## 北京市在售面积最小二手房

In [ ]:
house.sort_values('square').iloc[0,:]

## 北京市在售面积最大二手房

In [ ]:
house.sort_values('square',ascending=False).iloc[0,:]

## 各个行政区房源均价

In [ ]:
house_unitprice_perdistrict = house_detail.groupby('district').mean()['unitPrice']
house_unitprice_perdistrict.plot(kind='bar',x='district',y='unitPrice', title='各个行政区房源均价')
plt.legend(['均价']) 
plt.show()

## 各个区域房源数量排序

In [ ]:
bizcircle_count=house_detail.groupby('bizcircle').size().sort_values(ascending=False)
bizcircle_count.head(20).plot(kind='bar',x='bizcircle',y='size', title='各个区域房源数量分布')
plt.legend(['数量']) 
plt.show()

## 各个区域均价排序

In [ ]:
bizcircle_unitprice=house_detail.groupby('bizcircle').mean()['unitPrice'].sort_values(ascending=False)
bizcircle_unitprice.head(20).plot(kind='bar',x='bizcircle',y='unitPrice', title='各个区域均价分布')
plt.legend(['均价']) 
plt.show()

## 各个区域小区数量

In [ ]:
bizcircle_community=community.groupby('bizcircle')['title'].size().sort_values(ascending=False)
bizcircle_community.head(20).plot(kind='bar', x='bizcircle',y='size', title='各个区域小区数量分布')
plt.legend(['数量']) 
plt.show()

## 按小区均价排序

In [ ]:
community_unitprice = house.groupby('community').mean()['unitPrice'].sort_values(ascending=False)
community_unitprice.head(15).plot(kind='bar',x='community',y='unitPrice', title='各个小区均价分布')
plt.legend(['均价']) 
plt.show()